# Dropout丢弃法和Weight Decay的dist与理解

---

## 一、先搞懂"它们在控制什么"

### 权重衰退 vs Dropout 的本质区别

```
权重衰退：
  控制对象 → 权重 w 的大小
  怎么控制 → 更新 w 的时候，先把 w 缩小一点
  
  类比：
    w 就像汽车的油门踏板
    权重衰退说："别踩太深，踩 80% 就够了"
    
Dropout：
  控制对象 → 神经元的输出
  怎么控制 → 随机把一些输出变成 0
  
  类比：
    神经元就像团队里的成员
    Dropout 说："每次随机让一半人请假"
```

---

## 二、为什么位置不一样？

### 权重衰退在优化器里

**因为它改的是"更新权重的方式"！**

```python
# 没有权重衰退的更新：
w = w - lr × ∂L/∂w

# 有权重衰退的更新：
w = (1 - lr×λ) × w - lr × ∂L/∂w
    ↑
  先把 w 缩小一点！
  
这是"更新规则"的改变
→ 更新规则由"优化器"控制
→ 所以写在优化器里
```

**实际代码：**

```python
trainer = torch.optim.SGD(
    net.parameters(),
    lr=0.01,
    weight_decay=0.01  # ← 告诉优化器："更新时顺便把 w 缩小"
)

# 训练循环不用改
for X, y in train_iter:
    l = loss(net(X), y)  # 正常算损失
    trainer.zero_grad()
    l.backward()
    trainer.step()       # ← 优化器在这一步自动做权重衰退,⭐⭐⭐说白了就是更新参数
```

**为什么不在模型里？**

```
权重衰退不改变数据流向
它只改变"参数怎么更新"
数据还是正常地流过模型：
  输入 → Linear → ReLU → Linear → 输出
  
权重 w 在更新的时候才被缩小
不是在前向传播时缩小
```

---

### Dropout 在模型里

**因为它改的是"数据的流动"！**

```python
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Dropout(0.5),      # ← 在这里截断数据流
    nn.Linear(256, 10)
)
```

**数据流的变化：**

```
没有 Dropout：
  输入 [256, 784]
    ↓ Linear
  [256, 256]
    ↓ ReLU
  [256, 256] = [0.5, -0.3, 0.8, 0.2, -0.1, ...]
    ↓ 直接传给下一层
  下一层收到：[0.5, -0.3, 0.8, 0.2, -0.1, ...]

有 Dropout：
  输入 [256, 784]
    ↓ Linear
  [256, 256]
    ↓ ReLU
  [256, 256] = [0.5, -0.3, 0.8, 0.2, -0.1, ...]
    ↓ Dropout 把一半变成 0，剩下的 ×2
  [256, 256] = [1.0,  0,  1.6,  0,  -0.2, ...]
    ↓ 下一层收到的是被修改过的数据
  下一层收到：[1.0, 0, 1.6, 0, -0.2, ...]  ← 不一样！
```

**Dropout 像一个"过滤器"：**

```
Linear → ReLU → [Dropout 在这截断] → Linear
                  ↑
            数据流过这里时被改变
            必须放在数据流动的路径上
            → 所以在模型里
```

---

## 三、用生活例子对比

### 权重衰退 = 限制工具的力度

```
你有一把电钻（模型）
钻头的转速（权重 w）可以调

权重衰退说：
  "转速别开太猛，开到 80% 就够了"
  
这是在调整"工具的使用方式"
→ 对应"优化器"（控制怎么更新参数）

你不需要在电钻上加零件
只需要改变"怎么用它"
```

### Dropout = 随机拆掉一些零件

```
你有一辆车（模型）
车上有 4 个轮子（4 个神经元）

Dropout 说：
  "每次开车前，随机卸掉 2 个轮子"
  
这是在改变"车的结构"
→ 对应"模型本身"（改变数据流）

你必须真的在车上动手脚
不能只是"改变开车方式"
```

---

## 四、为什么预测不用 Dropout？

### 用最简单的例子

假设你是一个学生（模型），要参加考试（预测）。

#### 训练阶段（平时学习）

```
老师用 Dropout 训练你：

第1天：
  老师说："今天不许用左手"
  你只能用右手写字、吃饭、做题
  → 你学会了"不依赖左手也能学习"

第2天：
  老师说："今天不许用右手"
  你只能用左手
  → 你学会了"不依赖右手也能学习"

第3天：
  老师说："今天两只手都能用，但不许用左眼"
  → 你学会了"不依赖某个特定能力"

...

结果：
  你变成了"全能选手"
  不会过度依赖某一个技能
```

#### 预测阶段（最后的考试）

```
考试的时候：

❌ 错误做法（继续用 Dropout）：
  考试时老师说："继续随机禁用你的某些能力"
  → 你：？？？我要考试啊！
  → 有时候左手被禁，有时候右手被禁
  → 每次考试成绩都不一样（随机的）
  → 这不合理！

✅ 正确做法（不用 Dropout）：
  考试时老师说："发挥你的全部能力吧"
  → 你用上所有技能
  → 因为你平时被强制"不依赖某个技能"训练过
  → 现在用全部技能，效果更好
  → 每次考试成绩稳定
```

### 用代码对比

```python
# 训练时（用 Dropout）
net.train()  # 开启训练模式

for X, y in train_iter:
    output = net(X)
    # 每次前向传播，Dropout 都随机关掉一些神经元
    # 第1批：关掉 [1, 3, 5]
    # 第2批：关掉 [2, 4]
    # 第3批：关掉 [1, 2, 3]
    # ...
    
    # 这让模型学会"不依赖某些固定的神经元"
```

```python
# 预测时（不用 Dropout）
net.eval()  # 开启评估模式

output = net(X_test)
# 所有神经元都工作
# 每次输入同样的 X，输出都一样（确定性）
```

----

### 更直观的例子

```
训练：
  就像你投篮练习
  教练让你：
    第1次只用左手投 10 次
    第2次只用右手投 10 次
    第3次只用左手投 10 次
    ...
  
  这让你左右手都练得很均衡

比赛（预测）：
  你当然要用双手投篮！
  不会傻到继续"只用左手"或"只用右手"
  
  用双手 = 训练时"左手"和"右手"效果的平均
```

---

## 完整对比表

| | 权重衰退 | Dropout |
|---|---|---|
| **控制什么** | 权重 w 的大小 | 神经元的输出 |
| **怎么控制** | 更新 w 时先缩小 | 前向传播时随机置 0 |
| **改变什么** | 参数更新方式 | 数据流动 |
| **写在哪里** | 优化器里 | 模型里 |
| **训练时** | 每次更新 w 都缩小 | 每次前向传播都随机 |
| **预测时** | 不做任何事 | 不做任何事 |
| **为什么预测不用** | 权重已经训练好了 | 要确定的输出，不能随机 |

---

## 代码完整示例

```python
# ========== 搭建模型 ==========
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Dropout(0.5),      # ← Dropout 在这（模型里）
    nn.Linear(256, 10)
)

# ========== 创建优化器 ==========
trainer = torch.optim.SGD(
    net.parameters(),
    lr=0.01,
    weight_decay=0.01     # ← 权重衰退在这（优化器里）
)

# ========== 训练 ==========
net.train()  # 开启 Dropout
for X, y in train_iter:
    l = loss(net(X), y)
    trainer.zero_grad()
    l.backward()
    trainer.step()        # 权重衰退在这一步生效

# ========== 预测 ==========
net.eval()   # 关闭 Dropout
output = net(X_test)      # 得到确定的输出
```

---

## 自查题

```
① 权重衰退写在哪里？为什么？
   → 优化器里，因为它改变"参数更新方式"

② Dropout 写在哪里？为什么？
   → 模型里，因为它改变"数据流动"

③ 预测时用权重衰退吗？
   → 不用，权重已经训练好了

④ 预测时用 Dropout 吗？
   → 不用，要确定的输出

⑤ 如果预测时继续用 Dropout 会怎样？
   → 每次输出都不一样（随机的），不合理
```

---
